# Intialisation of the process on Google Colab

In [ ]:
# Installation of Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# Cloning of the GitHub repository onto Google Drive
! git clone https://github.com/GSacchetti/Segmentation-of-multispectral-and-thermal-images-with-U-NET.git

In [ ]:
# Access to the folder on google Drive, if it doesn't work, access the pannel on the left, search for the repository and copy the path
%cd gdrive/My Drive/U-NET

In [ ]:
# See all the folders contained in the repository
!ls

In [ ]:
# Installation of the requirements
!pip install -r requirements.txt

In [ ]:
# Install a new version of xlrd
!pip install xlrd==1.2

In [ ]:
# Verify that the right version has been installed (1.2.0)
import xlrd
print(xlrd.__VERSION__)


# Initialisation of the GPU



To make sure you're borrowing a GPU from Google :
  Go to "Edit" -> "Notebook settings" -> "Hardware accelerators" change it from "None" to "GPU" and finally "Save". 
  
  Once this is done, run the three following cells to make sure the GPU is working correctly

In [ ]:
# This cell return the name of the GPU (/device:GPU:0)
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
!cat /proc/meminfo

# Data Preparation

In [ ]:
# See the code for the transformation from excel to tif
!cat Construction_data/excel_to_tif_array.py

In [ ]:
# Run the code to create the tif images
%run Construction_data/excel_to_tif_array.py

In [ ]:
# See the code of create_train_test_data
!cat Construction_data/create_train_test_data.py

In [ ]:
# Split of the dataset 80% for training and 20% for verification
%run Construction_data/create_train_test_data.py

# Training of U-NET

In [ ]:
# See the function for loading the training and test datasets
!cat load_train_test_data.py

In [ ]:
# See the model for the U-NET method, in this file the learning rate, optimizer, loss function, number of epochs before early stopping
!cat model_unet.py

In [ ]:
# See the training algorithm for U-NET, in this file, it is possible to change the channels taken into account for the learning, as well as if it should load the weight or create
# a new weight file
!cat main.py

In [ ]:
# Run the training algorithm, it will first load the datasets, then summarize the model and finally start training.
# To change the channels used simply add : "[4,5]" after %run main.py.
# Example :
#%run main.py "[0,1,2]"
%run main.py

In [ ]:
# See the pred_unet file to make prediction depending on the weight file
!cat pred_unet.py

IMPORTANT : Make sure the channels indicated in that file are the same ones as the one used for training.

Channels can be changed directly in the file as well as the weight file to use.

Multiple weight files are provided, trained on different combination of channels.

To do so, go to line 63 :
model = load_model('model-U_NET.h5')
and change 'model-U_NET.h5' by 'model-U_NET_[...].h5', with in the brackets the channels wanted. Check "!ls" at the beginning to review all the available weight files..

In [ ]:
# To change the channels used simply add : "[4,5]" after %run pred_unet.py, also it is still needed to change the name of the weight file in the script according
# to the restult you want.
# Example :
#%run pred_unet.py "[0,1,2]"

# To make a prediction on one subset, use %run pred_unet.py "[0,1,2,3,4,5,6]" "npy" or %run pred_unet.py "[0,1,2,3,4,5,6]"
# To make predictions on the full Dalles contained in "/content/gdrive/My Drive/U-NET/data/Image_to_pred/DallesTif", use %run pred_unet "[0,1,2,3,4,5,6]" "tif"
%run pred_unet.py

# Precision measurment 

In [ ]:
# Loading of the images and transformation in array

from PIL import Image
from os.path import join
import numpy as np

# Ground truth :
tif_path = join('/content/gdrive/My Drive/U-NET/data/Image_to_pred', 'Image_pred.tif')
im_truth = Image.open(tif_path)

truth_array = np.array(im_truth)

# Latest Prediction :
pred_path = join('/content/gdrive/My Drive/U-NET/EVALUATIONS','0.5Image.npy.png')
im_pred = Image.open(pred_path)

pred_array = np.array(im_pred)

In [ ]:
truth_array_0 = truth_array[:,:,0]
truth_array_1 = truth_array[:,:,1]
truth_array_2 = truth_array[:,:,2]
modif_array_0 = modif_array[:,:,0]

In [ ]:
# Precision formula :
(np.sum(pred_array==modif_array_0)/truth_array_0.size)*100

In [ ]:
# See the measurment_functions file
!cat measurment_functions.py

In [ ]:
# Returns nb_True_Negative, nb_True_Positive, nb_False_Positive, nb_False_Negative, accuracy, precision, recall, f_beta_score
import measurment_functions
beta = 1 # Change beta in function of the wanted results
measurment_functions.measure(beta, pred_array, truth_array_0)